- Preparar cuenta y CLI (suscripción activa). 

- Crear Resource Group.

- Crear VNet + subredes (y NSG básico). 


- Crear Key Vault (modelo RBAC recomendado) y ajustar permisos. 


- Crear grupos en Entra ID y asignar roles (acceso a Key Vault / RG). 


- Desplegar VM en la VNet, habilitar Managed Identity y dar permiso al identity sobre Key Vault (prueba). 

- Opcional avanzado: proteger Key Vault con Private Endpoint (Private Link).

- az login -> inciar sesión en AZure CLI
- az account list --output table ->

### crear un grupo de recursos para hospedar la red virtual:
- az group create --name rg-data-lab --location eastus
- az group list --output table -> listar los grupos de recursos creados

#### Crear una VNet (red viertaul) y una subred en el grupo de recursos gr-data-lab:
- az network vnet create --resource-group rg-data-lab --name vnet-data-lab --address-prefixes 10.10.0.0/16 --subnet-name subnet-app --subnet-prefixes 10.10.1.0/24 --location eastus

#### verificar lista de VNet y subred creadas en el recurso 
- az network vnet list --resource-group rg-data-lab --output table
- az network vnet subnet list --resource-group rg-data-lab --vnet-name vnet-data-lab --output table

#### Crear un Network Security Group (NSG)

Crear un Network Security Group (NSG) y asociarlo a tu subred te permite controlar el tráfico de red (inbound/outbound), actuando como un firewall a nivel de red dentro de Azure.

- az network nsg create --resource-group rg-data-lab --name nsg-app

#### listar/ ver detalles de los NSG 
 - az network nsg list --resource-group rg-data-lab --output table

 - az network nsg show --resource-group rg-data-lab --name nsg-app --output json

#### asociar una nsg a un subred
 az network vnet subnet update --resource-group rg-data-lab --vnet-name vnet-data-lab --name subnet-app --network-security-group nsg-app

### Crear Key Vault (modelo RBAC recomendado)
¡¡¡el nombre del Key Vault debe ser único a nivel global (solo letras y números, sin mayúsculas)!!! 

-- se debe registrar el proveedor:
- az provider register --namespace Microsoft.KeyVault
- az provider show --namespace Microsoft.KeyVault --query "registrationState" --output tsv

ahora si se pueden crer las key vault:

-  az keyvault create --name kv-datalab2025 --resource-group rg-data-lab --location eastus --sku standard

#### listar las key vault existentes
- az keyvault list --resource-group rg-data-lab --output table
- az keyvault show --name kv-datalab2025 --resource-group rg-data-lab -o table

#### asigar un rol a un usuario
obtener el id en Entra:
az ad signed-in-user show --query id -o tsv

#### crear la asignacion
 az role assignment create --assignee da32158b-55e2-429d-80ea-8369b5ed768e --role "Key Vault Secrets Officer" --scope $(az keyvault show --name kv-datalab2025 --query id -o tsv)

#### Guardar secretos en las key vault 
 az keyvault secret set \
  --vault-name kv-datalab2025 \
  --name "DbPassword" \
  --value "SuperP@ssw0rd123"